# Symmetry analysis

In order to compute phonons and their interactions, symmetry analysis needs to be performed to determine the irreducible derivtives that are contained in the give system size.
Using irreducible derivtives guarantees the symmetry of the interaction coefficients as well as computation efficiency.

Here we perform symmetry analysis in reciprocal space, maximally exploit space group symmetry of the crystal.
The process is divided in several steps, for a given system size (FTG) and interaction order:
1. We find the irreducible Q-points fits the FTG, at the given order.
2. For each Q-point, we find irreducible derivatives allowed by symmetry.

## Irreducible Q-points

In the translation group module, we have discussed finding irreducible q-points within a given supercell with symmetry.
In reciprocal space, an N-th order phonon interaction is denoted by a collection of N q-points (here we refer to as a single Q-point), that have to add to a reciproal lattice vector due to conservation of momentum.
The task at hand becomes at given order, finding the irreducible set of Q-points that we need to compute the phonon interactions for.

Since the displacements of nuclei commute with the BO energy surface, these energy derivatives are invariant to the order of how the derivative is taken. This is the permutation symmetry of the derivatives.

We are able to find the irreducible Q-points using the space group symmetry of the crystal and permutation symmetry of derivatives. All the other Q-points can be computed from this irreducible set through rotation and trapose.


For example, we take a look at the FCC structure with Oh symmetry at 3rd order in $2\hat{1}$ supercell.

In [1]:
import numpy as np
from principia_materia.translation_group.qpoints_n import QpointsN

In [2]:
vec = np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])
supa = np.array([
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 2],
])
qpoints_n = QpointsN(
    vec=vec,
    supa=supa,
    order=3,
    pg="Oh",
)

The algorithm is design to find the permutationally irreducible Q-points, then use the space group symmetry and permutation map to find the irreducible Q-points.

Here we can see the Qpoints get reduced due to the 2 types of symmetry step by step.

In [3]:
len(qpoints_n.Qpoints), \
    len(qpoints_n.Qpoints_perm.irreducible_Qpoints), \
    len(qpoints_n.irreducible_Qpoints)


(64, 15, 5)

## Irreducible derivatives at a given Q-point - little group approach

Little group approach perform symmetry analysis with the little group of the Q-point, which is the space group leaves each q-point of the Q-point invariant. 
This is the most straightforward approach, but doesn't not exploit the full potential of the space group.

The symmetry analysis consists of following steps:
1. Find the little group of the Q-point
2. Find the irreducible representations of the displacements at each q-point
3. Use symmetry direct product group to determine the irreducible derivatives.

In [4]:
from principia_materia import Fraction
from principia_materia.io_interface import parse_array
from principia_materia.io_interface.vasp import parse_poscar
from principia_materia.translation_group import get_structure, CrystalFTG
from principia_materia.phonon_id.little_group_adt import LittleGroupADT

Here we check out an example with rocksalt structure at $(\Gamma, L_{a}, L_{a})$ Q-point, where 5 irreducible derivatives are found.

In [5]:
structure = get_structure(parse_poscar("nacl/POSCAR"), stype=CrystalFTG)
structure.orbitals = "p"

In [43]:
Qpoint = parse_array("0 0 0; 1/2 0 0; 1/2 0 0", dtype=Fraction)
Qpoint

[[Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)],
 [Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)],
 [Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)]]

In [44]:
lgadt = LittleGroupADT(
    structure=structure,
    pg="Oh",
    Qpoint=Qpoint)
lgadt.set_qpoint_displacement_rep()
lgadt.set_irreducible_derivatives()
lgadt.set_vectorized_tensor()

In [48]:
lgadt.n_irreducible_derivatives

5

In [45]:
lgadt.irreducible_derivative_names

[((((Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)), ('A2u', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('A1g', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('A2u', 0))),
  ('A1g', 0)),
 ((((Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)), ('A2u', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('Eg', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('Eu', 0))),
  ('A1g', 0)),
 ((((Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)), ('Eu', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('A1g', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('Eu', 0))),
  ('A1g', 0)),
 ((((Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)), ('Eu', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('A2u', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('Eg', 0))),
  ('A1g', 0)),
 ((((Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)), ('Eu', 0)),
   ((Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)), ('Eg', 0)),
   ((F

The vectorized tensors are tensors that each element of them is a vector representing 
the linear combination of the irreducible derivatives. 
This is an efficient way to store the analytic dynamic tensor in terms of irreducible derivatives.

The dynamic tensor has shape of (6, 6, 6) and there are 5 irreducible derivatives, thus the analytic tensor has shape of (6, 6, 6, 5).

In [47]:
lgadt.vectorized_tensor.shape

(6, 6, 6, 5)

In [26]:
structure.to_dict()

OrderedDict([('vec',
              array([[0.        , 2.83000729, 2.83000729],
                     [2.83000729, 0.        , 2.83000729],
                     [2.83000729, 2.83000729, 0.        ]])),
             ('atoms',
              OrderedDict([('Na', array([[0., 0., 0.]])),
                           ('Cl', array([[0.5, 0.5, 0.5]]))])),
             ('orbitals', 'p'),
             ('supa',
              array([[1, 0, 0],
                     [0, 1, 0],
                     [0, 0, 1]]))])

## Symmetry analysis in a FTG

Now that we have discussed the 2 steps of the symmetry analysis.
Combine them together and we can compute the vibrational hamiltonian of a given FTG and order in terms of irreducible derivatives.

We can use this information to write out the hamiltonian to execute some back-of-envelope calculations, or we can perform more complicated analysis. For example, we can compute the chainrule matrices to determine how to use the minimum amount of displacements to probe the entire system, which is the core feature of the BID approach, or we can Fourier transform the analytic tensors into real space force tensors for further analysis.

Here we will discuss the chainrule analysis that is used in BID approach.

## Chainrule derivatives

After we obtain the analytic tensors of a given FTG and order, we can compute the chainrule matrices, that represent the contribution of the irreducible derivatives to the derivative from the provided displacement basis.
This allows us to convert between derivatives of displacement basis of choice and the irreducible derivatives, which is the key to computing phonon interaction coefficients using finite displacements.

Here, we use the same rocksalt example to compute the chainrule matrix of an arbitrary displacement.

In [49]:
from principia_materia.phonon_id.chainrule_derivatives import ChainruleDerivatives

In [51]:
structure = get_structure(parse_poscar("nacl/POSCAR"), stype=CrystalFTG)
structure.orbitals = "p"
supa = np.array([
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 2],
])

In [68]:
chainrule =  ChainruleDerivatives(
    structure=structure,
    supa=supa,
    order=3,
    pg='Oh')
chainrule.set_ADT(verbose=True)
chainrule.set_basis()

Computing ADT for Qpoint: ((0, 0, 0), (0, 0, 0), (0, 0, 0))
Computing ADT for Qpoint: ((1/2, 0, 0), (1/2, 0, 0), (0, 0, 0))
Computing ADT for Qpoint: ((1/2, 1/2, 0), (1/2, 1/2, 0), (0, 0, 0))
Computing ADT for Qpoint: ((1/2, 1/2, 0), (0, 1/2, 0), (1/2, 0, 0))
Computing ADT for Qpoint: ((0, 1/2, 1/2), (1/2, 0, 1/2), (1/2, 1/2, 0))
Summarize irreducible derivatives.
Compute vectorized tensors.


In [69]:
displacements = np.zeros((1, chainrule.order - 1, chainrule.supercell.natoms, chainrule.supercell.dim))
displacements[0, 0, 0, 0] = 1
displacements[0, 1, 1, 0] = 1

In [70]:
matrix = chainrule.compute_chainrule(displacements)

In [71]:
matrix.shape

(1, 8, 6, 61)